### 代码信息
    版本: 1.2
    作者: xdxt
    创建日期: 2020/3/15
    更新人员: xdxt
    更新日期: 2020/3/21
    描述: 构建 resnet18卷积神经网络模型对10-monkey-species数据集分类

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import os
import numpy as np
import pandas as pd 
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


### 路径配置

In [ ]:
MONKEY_DATA_DIR = "./data/10-monkey-species"
train_dir = os.path.join(MONKEY_DATA_DIR, "training")
valid_dir = os.path.join(MONKEY_DATA_DIR, "validation")
label_file_dir = os.path.join(MONKEY_DATA_DIR, "monkey_labels.txt")

print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file_dir))

labels = pd.read_csv(label_file_dir, header=0)
print(labels)

### 工具函数

In [ ]:
def make_folder(folder_list):
    """if folder isn't exists then make folder"""
    for folder_path in folder_list:
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
            
            
def show_image(img, title=''):
    """show image"""
    # image_data.transpose(1, 2, 0) # 32, 32, 3
    plt.title(title, color='red')
    plt.imshow(img)
    plt.axis('off')#不显示坐标值
    plt.show()


def plot_learning_curves(history, labels=[], min_value=0, max_value=1, epochs=None, save_path=None):
    """dict to chart image"""
    info = {}
    if labels == []:
        labels = [k for k, v in history.history.items()]
    for b in labels:
        info[b] = history.history[b]
    if epochs is None:
        epochs = len(info[labels[0]])
        
    pd.DataFrame(info).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    if save_path is not None:
        plt.savefig(save_path + ".jpg")
    plt.show()

### 数据增强配置与分割
#### 数据集1370共张图片按如下划分：
    272  -> 验证集
    1098 -> 训练集

In [ ]:
"""
图片尺寸配置[把规格不一的图片数据处理成以下配置]
"""

# 宽度
height = 224
# 长度
width = 224
# 通道数[rgb]
channels = 3
# 对少个一组
batch_size = 48
# 类别数目
num_classes = 10

# 读取并修改图像
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input, # 自带归一
    rotation_range = 40, # 对图片随机旋转(-40, 40)度
    width_shift_range = 0.2, # 对图片做随机(0%, 20%)水平位移,小于1代表移动比例
    height_shift_range = 0.2, # 对图片做随机垂直位移,大于1代表移动像素
    shear_range = 0.2, # 剪切强度
    zoom_range = 0.2, # 缩放程度
    horizontal_flip = True, # 启动随机水平翻转
    fill_mode = 'nearest' # 图像放大时，按就近像素点模拟填充
)
# 读取图片后按上面修改，再转成规定尺寸
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = "categorical") # 控制label格式，这里设为one_hot过的[0,0,1,0]; 还有不one_hot的index型第1类[0], 第6类[5]


valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = False,
                                                   class_mode = "categorical")

train_num = train_generator.samples
valid_num = valid_generator.samples

### 构建模型结构

In [ ]:
def resnet_conv2D(input_x, filters, kernel_size=3, strides=1, padding='same', activation=None):
    input_x = keras.layers.Conv2D(filters=filters, 
                                  kernel_size=kernel_size, 
                                  strides=strides,
                                  padding=padding)(input_x)
    input_x = keras.layers.BatchNormalization()(input_x)
    if activation is not None:
        input_x = keras.layers.Activation(activation=activation)(input_x)
    return input_x


def identity_block(input_x, filters, kernel_size=3, strides=1, padding='same', activation=tf.nn.relu):
    shortcut = input_x
    input_x = resnet_conv2D(input_x, filters, kernel_size=kernel_size, 
                            strides=strides, padding=padding, activation='relu')
    input_x = resnet_conv2D(input_x, filters, kernel_size=kernel_size, 
                            strides=1, padding=padding)
    if(shortcut.shape[3] is not input_x.shape[3]):
        shortcut = resnet_conv2D(shortcut, input_x.shape[3],
                                 kernel_size=1, strides=strides)    
    
    input_x = keras.layers.Add()([input_x, shortcut])
    input_x = keras.layers.Activation(activation=activation)(input_x)
    return input_x

In [ ]:
"""
resnet-18
"""
input_x = keras.layers.Input((height, width, channels))
# conv1
output = keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(input_x)
output = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(output)
# conv2
output = identity_block(output, 64)
output = identity_block(output, 64)
# conv3
output = identity_block(output, 128, strides=2)
output = identity_block(output, 128)
# conv4
output = identity_block(output, 256, strides=2)
output = identity_block(output, 256)
# conv5
output = identity_block(output, 512, strides=2)
output = identity_block(output, 512)
# conv average pooling
output = keras.layers.AveragePooling2D(pool_size=2)(output)
output = keras.layers.Flatten()(output)
output = keras.layers.Dense(num_classes, activation = 'softmax')(output)

model = keras.models.Model(inputs=input_x, outputs=output, name='ResNet-18')

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam", 
              metrics = ["accuracy"])

model.summary()

### callbacks 配置

In [ ]:
base_path = os.path.join("callbacks", "10-monkey-species-callbacks")
logdir = os.path.join(base_path, "4.2-3-10-monkeys-resnet18_model")
make_folder([base_path, logdir])

output_model_file = os.path.join(logdir, "10-monkey-species_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(
        output_model_file, 
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
    ),
    keras.callbacks.EarlyStopping(patience=50, min_delta=1e-4),
]

In [ ]:
epochs = 150
# steps_per_epoch: 由于每轮数据是一直渲染不同的，所以得知道一轮数据会经历几次batch_size
history = model.fit(train_generator, 
                    steps_per_epoch=train_num // batch_size,
                    epochs=epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_num // batch_size,
                    callbacks=callbacks)

### 训练结果可视化

In [ ]:
image_folder = os.path.join(base_path, "compare-image")
make_folder([image_folder])
image_path = os.path.join(image_folder, time.strftime("%d_%H_%M_%S", time.localtime()))

plot_learning_curves(history, labels=['accuracy', 'val_accuracy'], min_value=0.8, save_path=image_path+"-1")
plot_learning_curves(history, labels=['loss', 'val_loss'], min_value=0, max_value=0.6, save_path=image_path+"-2")